In [19]:
# !pip install pandas
# !pip install datasets


In [1]:
import os

os.environ["HF_HOME"] = "/home/ec2-user/SageMaker/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/home/ec2-user/SageMaker/huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = "/home/ec2-user/SageMaker/huggingface_cache"
print("Cache redirected to larger volume.")



Cache redirected to larger volume.


In [11]:
os.listdir("translated_subsets")

['translated_wikiqa_train_sample.json',
 'translated_cnn_dailymail_train_sample.json',
 'translated_nqopen_train_sample.json',
 'translated_flan_qa_train_sample.json',
 'translated_flan_lambada_train_sample.json',
 'translated_xlel_wd_train_sample.json',
 'translated_wiki_split_train_sample.json',
 'translated_joke_explaination_train_sample.json',
 'translated_adversarial_qa_train_sample.json',
 'translated_dolly_train_sample.json',
 'translated_hotpotqa_train_sample.json',
 'translated_flan_coqa_train_sample.json',
 'translated_paws_train_sample.json',
 'translated_flan_gem_wiki_train_sample.json',
 'translated_mintaka_train_sample.json',
 'translated_soda_train_sample.json',
 'translated_flan_cot_train_sample.json',
 'translated_piqa_train_sample.json']

In [12]:
translated_subsets

['translated_adversarial_qa',
 'translated_cnn_dailymail',
 'translated_dolly',
 'translated_flan_coqa',
 'translated_flan_cot',
 'translated_flan_gem_wiki',
 'translated_flan_lambada',
 'translated_flan_qa',
 'translated_hotpotqa',
 'translated_joke_explaination',
 'translated_mintaka',
 'translated_mlqa',
 'translated_nqopen',
 'translated_paws',
 'translated_piqa',
 'translated_soda',
 'translated_wiki_split',
 'translated_wikiqa',
 'translated_xlel_wd']

In [2]:
all_subset_names = ['aya_dataset', 'templated_afriqa', 'templated_afrisenti', 'templated_amharic_qa', 'templated_armenian_instruct', 'templated_bengali_news', 'templated_dutch_imdb', 'templated_hindi_headline', 'templated_hindi_news', 'templated_indic_paraphrase', 'templated_indic_sentiment', 'templated_indo_stories', 'templated_japanese_instruct', 'templated_joke_explaination', 'templated_ligurian_news', 'templated_masakhanews', 'templated_mintaka', 'templated_ntx_llm', 'templated_nusax_senti', 'templated_persian_farstail', 'templated_persian_instruct', 'templated_scirepeval', 'templated_seed_instruct', 'templated_soda', 'templated_tamil_stories', 'templated_tamil_thirukkural', 'templated_telugu_food', 'templated_telugu_jokes', 'templated_telugu_news', 'templated_telugu_poems', 'templated_telugu_riddles', 'templated_thai_pos', 'templated_thai_scb', 'templated_thai_usembassy', 'templated_thai_wikitionary', 'templated_turku_paraphrase', 'templated_ukranian_gec', 'templated_uner_llm', 'templated_urdu_news_category', 'templated_urdu_news_gen', 'templated_urdu_news_headline', 'templated_wiki_split', 'templated_xcsqa', 'templated_xlel_wd', 'templated_xwikis', 'translated_adversarial_qa', 'translated_cnn_dailymail', 'translated_dolly', 'translated_flan_coqa', 'translated_flan_cot', 'translated_flan_gem_wiki', 'translated_flan_lambada', 'translated_flan_qa', 'translated_hotpotqa', 'translated_joke_explaination', 'translated_mintaka', 'translated_mlqa', 'translated_nqopen', 'translated_paws', 'translated_piqa', 'translated_soda', 'translated_wiki_split', 'translated_wikiqa', 'translated_xlel_wd']
templated_subsets = [s for s in all_subset_names if "templated" in s]
translated_subsets = [s for s in all_subset_names if "translated" in s]
aya_subset = ["aya_dataset"]

In [3]:
len(all_subset_names), len(templated_subsets), len(translated_subsets), len(aya_subset)

(64, 44, 19, 1)

### Sampling Code
Note: We will use the following num_samples for each subset to get a stratified sample by subset name for each partition (aya, templated, translated)
```
templated_subsets --> 114 (44 subsets, we want a 5k sample 5000/44)
translated_subsets --> 264 (19 subsets, we want a 5k sample 5000/19)
aya_subset --> 5000 (1 subsets, we want a 5k sample 5000/1)


In [14]:
from datasets import load_dataset
import random
import os
import json
import gc  # For explicit garbage collection

def sample_from_dataset(subset_name, output_dir, num_samples=264):
    """
    Sample from a single dataset subset and save to disk.
    """
    print(f"\nProcessing subset: {subset_name}")
    output_path = os.path.join(output_dir, f"{subset_name}_train_sample.json")
    
    try:
        # Load only the train split
        dataset = load_dataset("CohereForAI/aya_collection", subset_name, split="validation", download_mode="reuse_dataset_if_exists")  # Select only the first `num_samples` rows

        
        # Calculate indices for sampling
        total_size = len(dataset)
        if total_size < num_samples:
            print(f"Dataset has fewer than {num_samples} rows; sampling all {total_size} rows.")
            indices = range(total_size)
        else:
            # Set the random seed
            random.seed(seed)
            indices = random.sample(range(total_size), num_samples)
        
        # Sample using indices
        sampled_data = [dataset[idx] for idx in indices]
        
        # Save to disk
        with open(output_path, "w", encoding='utf-8') as f:
            json.dump(sampled_data, f, ensure_ascii=False, indent=2)
        
        print(f"Successfully sampled {len(sampled_data)} entries from {subset_name}")
        print(f"Saved to: {output_path}")
        
    except Exception as e:
        print(f"Error processing {subset_name}: {str(e)}")
    
    finally:
        # Explicit cleanup
        if 'dataset' in locals():
            del dataset
        if 'sampled_data' in locals():
            del sampled_data
        gc.collect()  # Force garbage collection
        
# Create output directory
output_dir = "templated_subsets"
os.makedirs(output_dir, exist_ok=True)

for subset_name in templated_subsets:
    sample_from_dataset(subset_name, output_dir, num_samples=114)

print("\nAll subsets processed.")


Processing subset: translated_mlqa
Successfully sampled 264 entries from translated_mlqa
Saved to: translated_subsets/translated_mlqa_val_sample.json

All subsets processed.


In [15]:
import pandas as pd
import os
import json

# Directory containing the sampled JSON files
input_dir = "templated_subsets"

# List to store data from all JSONs
all_data = []

# Iterate through all JSON files in the directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".json"):
        subset_name = file_name.rsplit(".", 1)[0]  # Get the file name without the extension
        file_path = os.path.join(input_dir, file_name)
        
        # Load the JSON file
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            
        # Add a column for subset_name
        for entry in data:
            entry["subset_name"] = subset_name
        
        # Append to the all_data list
        all_data.extend(data)

# Create a pandas DataFrame from the combined data
df = pd.DataFrame(all_data)

In [16]:
# Save the DataFrame to a Parquet file without the index
output_file = "templated_sample_5k.parquet"
df.to_parquet(output_file, index=False)

print(f"DataFrame successfully saved to {output_file}")


DataFrame successfully saved to translated_sample_5k.parquet


### Concatenating Subsets

In [18]:
# File paths
templated_file = "aya-data-quality/templated_sample_5k.parquet"
translated_file = "aya-data-quality/translated_sample_5k.parquet"
aya_file = "aya-data-quality/aya_sample_5k.parquet"

# Load datasets
templated_df = pd.read_parquet(templated_file)
translated_df = pd.read_parquet(translated_file)
aya_df = pd.read_parquet(aya_file)

# Add 'aya_partition' column
templated_df["aya_partition"] = "templated"
translated_df["aya_partition"] = "translated"
aya_df["aya_partition"] = "aya"

# Concatenate datasets
combined_df = pd.concat([templated_df, translated_df, aya_df], ignore_index=True)



In [19]:
combined_df

,id,inputs,targets,dataset_name,sub_dataset_name,task_type,template_id,language,split,script,subset_name,aya_partition
0,303,Quæ a l’é a traduçion italiaña de sto testo?\n...,A traduçion in italian do testo a l’é:\nSalari...,Lijnews-instruct,lijnews-instruct-lij-ita,translation,18,lij,train,Latn,templated_ligurian_news_train_sample,templated
1,212,Traduci questo testo in genovese:\nL'Europa ap...,La traduzione in genovese del testo è:\nL'Eurö...,Lijnews-instruct,lijnews-instruct-ita-lij,translation,6,ita,train,Latn,templated_ligurian_news_train_sample,templated
2,170,Qual è la traduzione genovese di questo testo?...,La traduzione in genovese del testo è:\nAttent...,Lijnews-instruct,lijnews-instruct-ita-lij,translation,8,ita,train,Latn,templated_ligurian_news_train_sample,templated
3,290,Traduxi in lengua italiaña:\nPapê fäsci in scî...,A traduçion in italian do testo a l’é:\nFalsi ...,Lijnews-instruct,lijnews-instruct-lij-ita,translation,14,lij,train,Latn,templated_ligurian_news_train_sample,templated
4,331,Quæ a l’é a traduçion italiaña de sto testo?\n...,A traduçion in italian do testo a l’é:\nRussia...,Lijnews-instruct,lijnews-instruct-lij-ita,translation,18,lij,train,Latn,templated_ligurian_news_train_sample,templated
...,...,...,...,...,...,...,...,...,...,...,...,...
15071,151002,Ba mhaith liom tuirse a thabhairt do ghaolta a...,Ba mhaith liom a mholadh dóibh pictiúr greannm...,Aya-Dataset,-,-,0,gle,train,Latn,aya_dataset_train_sample,aya
15072,158437,Ερώτηση: Τι είναι η απάντηση στην αξία της ανά...,Απάντηση: Η ανάγνωση βιβλίων έχει αξία στην εκ...,Aya-Dataset,-,-,0,ita,train,Latn,aya_dataset_train_sample,aya
15073,65556,Haddii aan haysto 5 moos oo aan 2 xabo siiyo q...,Waxaad haysataa 3 moos.,Aya-Dataset,-,-,0,som,train,Latn,aya_dataset_train_sample,aya
15074,122083,Quelle est la capitale du Canada?,La capitale du Canada est Ottawa dans la provi...,Aya-Dataset,-,-,0,fra,train,Latn,aya_dataset_train_sample,aya


In [20]:
len(combined_df)

15076

In [21]:
# Save the DataFrame to a Parquet file without the index
output_file = "sampled_aya_all_subsets_15k.parquet"
combined_df.to_parquet(output_file, index=False)
